# MLN - Classificação de Swear Words

Modelo de Machine learning feito para identificar e classificar Xingamentos/ Swear Words em frases.

## Import de bibliotecas

In [4]:
# Instalação de mais bibliotecas
# Estão em ambas IAs:
!pip install kaggle
!pip install unidecode

In [5]:
# Etc:
import json
import random

# Manipulação dos dados:
import pandas as pd
import numpy as np

# Machine Learning:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# PLN (Processamento de Linguagem Natural) e Pré-Processamento de dados:
import unidecode
import nltk
from nltk import tokenize
from string import punctuation
from numpy.lib.function_base import vectorize
from sklearn.feature_extraction.text import CountVectorizer

## MLN - Classificação de "Swear Words"




In [6]:
with open('swear.json', 'r') as file:
    data = json.load(file)

In [7]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
frases_aleatorias = [
    "A vida é bela.",
    "O sol está brilhando hoje.",
    "O tempo voa quando você está se divertindo.",
    "A persistência leva ao sucesso.",
    "A criatividade é a chave para a inovação.",
    "Nunca é tarde demais para aprender algo novo.",
    "O conhecimento é poder.",
    "A amizade verdadeira é um tesouro.",
    "O trabalho em equipe gera grandes resultados.",
    "A felicidade está nas pequenas coisas da vida."
]

In [9]:
teste1 = []
teste2 = []
for i in range(300):
    frase = random.choice(frases_aleatorias)
    if random.randint(0, 1) == 1:
          palavra = random.choice(data["pt"])
          frase = frase + " " + palavra
          teste1.append(frase)
          teste2.append(1)
    else:
      teste1.append(frase)
      teste2.append(0)

In [10]:
df_swear = pd.DataFrame({"Frase":teste1, "Classificacao":teste2})

In [11]:
df_swear.head()

,Frase,Classificacao
0,O conhecimento é poder. chupa-mos,1
1,A felicidade está nas pequenas coisas da vida.,0
2,O conhecimento é poder.,0
3,A felicidade está nas pequenas coisas da vida.,0
4,A vida é bela.,0


In [12]:
df_swear["Classificacao"].value_counts()

0    164
1    136
Name: Classificacao, dtype: int64

In [13]:
vetorize2 = CountVectorizer(lowercase = False)
bow2 = vetorize2.fit_transform(df_swear["Frase"])

In [14]:
x_train, x_test, y_train, y_test = train_test_split(bow2, df_swear["Classificacao"], test_size = 0.2, random_state = 42)

In [15]:
lr = LogisticRegression()
lr.fit(x_train, y_train)
acuracia = lr.score(x_test, y_test)
print(acuracia)

0.75


In [16]:
#Tokenizador que separa palavras de simbolos como: . ! ?
tokenizer2 = tokenize.WordPunctTokenizer()

In [17]:
#Lista de simbolos
pontuacao = list()
for ponto in punctuation:
  pontuacao.append(ponto)

In [18]:
sem_acentos = [unidecode.unidecode(texto) for texto in df_swear["Frase"]]

In [19]:
df_swear["Frase2"] = sem_acentos

In [20]:
#Lista com as stopwords em português
palavras_irrelevantes = nltk.corpus.stopwords.words("portuguese")
pontuacao_stopwords = pontuacao + palavras_irrelevantes
#Retira os acentos das palavras
stopwords_sem_acento =  [unidecode.unidecode(texto) for texto in pontuacao_stopwords]

In [21]:
#Iterando na coluna do dataframe, colocando as palavras em minusculo, tokenizando, colocando em uma lista, iterando nessa lista, e verificando se são palavras relevantes ou não
frase_processada = list()
for frase in df_swear["Frase2"]:
    nova_frase = list()
    frase = frase.lower()
    palavras_texto = tokenizer2.tokenize(frase)
    for palavra in palavras_texto:
        if palavra not in stopwords_sem_acento:
            nova_frase.append(palavra)
    frase_processada.append(' '.join(nova_frase))

In [22]:
df_swear["Frase3"] = frase_processada

In [23]:
df_swear

,Frase,Classificacao,Frase2,Frase3
0,O conhecimento é poder. chupa-mos,1,O conhecimento e poder. chupa-mos,conhecimento poder chupa mos
1,A felicidade está nas pequenas coisas da vida.,0,A felicidade esta nas pequenas coisas da vida.,felicidade pequenas coisas vida
2,O conhecimento é poder.,0,O conhecimento e poder.,conhecimento poder
3,A felicidade está nas pequenas coisas da vida.,0,A felicidade esta nas pequenas coisas da vida.,felicidade pequenas coisas vida
4,A vida é bela.,0,A vida e bela.,vida bela
...,...,...,...,...
295,O trabalho em equipe gera grandes resultados. ...,1,O trabalho em equipe gera grandes resultados. ...,trabalho equipe gera grandes resultados cachorro
296,A amizade verdadeira é um tesouro.,0,A amizade verdadeira e um tesouro.,amizade verdadeira tesouro
297,A persistência leva ao sucesso.,0,A persistencia leva ao sucesso.,persistencia leva sucesso
298,A amizade verdadeira é um tesouro. caceta,1,A amizade verdadeira e um tesouro. caceta,amizade verdadeira tesouro caceta


In [24]:
bow3 = vetorize2.fit_transform(df_swear["Frase3"])

In [25]:
a_train, a_test, b_train, b_test = train_test_split(bow3, df_swear["Classificacao"], test_size = 0.2, random_state = 42)

In [26]:
#O resultado não mudou porque tem poucas palavras, mas é bom fazer o tratamento dos dados
lr = LogisticRegression()
lr.fit(a_train, b_train)
acuracia = lr.score(a_test, b_test)
print(acuracia)

0.75
